In [2]:
import re
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from plotly.subplots import make_subplots
# from folium import Map as fl_map
# from folium.plugins import HeatMap

In [3]:
# data = pd.read_csv('/content/drive/MyDrive/DHDS/properatti.csv')
data = pd.read_csv('data/properatti.csv')
data.shape

(121220, 26)

In [4]:
# Renombramos la columna lat-lon por generar errores en llamadas
data.rename(columns={'lat-lon': 'lat_lon'}, inplace=True)

# Renombramos las columnas "price_usd_per_m2 y price_per_m2" a "price_usd_surf_covered y price_usd_surf_total" para que representen mejor los datos contenidos.
data.rename(columns={'price_usd_per_m2'      : 'price_usd_surf_covered',
                     'price_per_m2'          : 'price_usd_surf_total',
                     'surface_total_in_m2'   : 'surface_total_m2',
                     'surface_covered_in_m2' : 'surface_covered_m2'}, inplace=True)
data.head(3)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat_lon,lat,...,surface_covered_m2,price_usd_surf_covered,price_usd_surf_total,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [5]:
# Creamos una copia del dataset original para no modificarla
data2 = data.copy()

# Eliminamos las columnas que no necesitamos
data2.drop(['Unnamed: 0', 'operation', 'expenses', 'properati_url', 'image_thumbnail'], axis=1, inplace=True)
data2[['property_type','state_name','place_name']] = data2[['property_type','state_name','place_name']].astype('category')

data2.head(3)

,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat_lon,lat,lon,price,...,price_aprox_local_currency,price_aprox_usd,surface_total_m2,surface_covered_m2,price_usd_surf_covered,price_usd_surf_total,floor,rooms,description,title
0,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,...,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB
1,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,...,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...
2,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,...,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO


---
# Gŕafico total de nulos por columna

In [6]:
data_nulls = pd.DataFrame(data.drop(['Unnamed: 0', 'operation', 'expenses', 'properati_url', 'image_thumbnail'], axis=1).isnull().sum().sort_values(ascending=False)[
                          data.drop(['Unnamed: 0', 'operation', 'expenses', 'properati_url', 'image_thumbnail'], axis=1).isnull().sum() > 0], columns=['Nulls'])
perc_nulls = pd.DataFrame(((data_nulls.values * 100)/data.shape[0]).round(3))
perc_nulls.columns = ['Percent']
perc_nulls = perc_nulls.applymap(lambda x: '{:.3f}%'.format(x))
data_nulls['Percent'] = perc_nulls.Percent.values


data2_nulls = pd.DataFrame(data2.isnull().sum().sort_values(ascending=False)[data2.isnull().sum() > 0], columns=['Nulls'])
perc_nulls = pd.DataFrame(((data2_nulls.values * 100)/data.shape[0]).round(3))
perc_nulls.columns = ['Percent']
perc_nulls = perc_nulls.applymap(lambda x: '{:.3f}%'.format(x))
data2_nulls['Percent'] = perc_nulls.Percent.values

px_bar = make_subplots(rows=1, cols=1)

px_bar.add_traces(go.Bar(name="Data Orignal", y=data_nulls.Nulls, x=data_nulls.index.values, text = data_nulls.Percent,
                         marker=dict(color = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], colorscale='Agsunset')))

px_bar.add_traces(go.Bar(name="Data2", y=data2_nulls.Nulls, x=data2_nulls.index.values, text = data2_nulls.Percent,
                         marker=dict(color = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], colorscale='Bluered')))


px_bar.update_xaxes(tickangle=45, tickfont=dict(size=10))
px_bar.update_layout(title_text = 'Nulos Original / Nulos post Data Wrangling', height=700)
px_bar.show()

---
# Evaluamos si existen filas duplicadas para las Features de interés

In [7]:
feature_cols = ['property_type','place_with_parent_names','currency','price','price_aprox_usd','price_aprox_local_currency',
                'surface_total_m2','price_usd_surf_total','surface_covered_m2','price_usd_surf_covered','description','title']

print(f"Registros duplicados:\n{data2.duplicated(subset=feature_cols, keep='first').value_counts()}\n\n")
print(f"Total de registros: {data2.shape[0]}")

Registros duplicados:
False    115678
True       5542
dtype: int64


Total de registros: 121220


Procedemos a eliminar los registros duplicados

In [8]:
data2.drop_duplicates(subset=feature_cols, keep='first', inplace=True)

Verificamos si se aplicaron los cambios

In [9]:
print(f"Registros duplicados:\n{data2.duplicated(subset=feature_cols, keep='first').value_counts()}\n\n")
print(f"Total de registros: {data2.shape[0]}")

Registros duplicados:
False    115678
dtype: int64


Total de registros: 115678


---
# Completando datos faltantes de la Feature "rooms"
Los datos de "rooms" serán extraidos de la Feature "description" mediante filtros de expresiones regulares y funciones lambda

In [10]:
# Creamos los patrones y compilamos
regex_rooms = re.compile(r'([1-9](\Wamb|dorm))')
regex_no_dig = re.compile('\D')

In [11]:
# Aplicamos los patrones mediante search unicamente sobre los registros nulos de "rooms"
rooms_desc = data2[["description"]][data2.rooms.isnull() == True].astype(str).applymap(lambda x: regex_rooms.search(x, re.IGNORECASE))

# Elimanamos los caracteres que no sean digitos
rooms_desc = rooms_desc[["description"]].applymap(lambda x: x if x is None else regex_no_dig.sub('', x[1]))

Una vez obtenida la información faltante de "rooms" de la Feature "description", debemos asignarla a data2["rooms"] usando index_mask

In [12]:
# Asiganoms los índices a una variable
indices_replace = rooms_desc[rooms_desc.values != None].index

# Asingamos los valores a data2["rooms"]
data2.loc[indices_replace, "rooms"] = rooms_desc.loc[indices_replace].values
data2[["rooms"]].isnull().value_counts()

rooms
True     58578
False    57100
dtype: int64

Verificamos si se aplicaron los cambios

In [13]:
print(f"Nulos original de feature 'rooms':                                {data.rooms.isnull().sum()}\n")
print(f"Total de registros de feature 'rooms':                            {data2.rooms.shape[0]}")
print(f"Nulos de 'rooms' después de asignar los datos obtenidos:          {data2.rooms.isnull().sum()}")
print(f"Porcentaje de nulos respecto al total de registros:               {round((data2.rooms.isnull().sum()*100)/data2.shape[0], 2)}%")

Nulos original de feature 'rooms':                                73830

Total de registros de feature 'rooms':                            115678
Nulos de 'rooms' después de asignar los datos obtenidos:          58578
Porcentaje de nulos respecto al total de registros:               50.64%


Puesto que el porcentaje de nulos sobre "rooms" sigue siendo muy alto, es una variable que será reservada solo para fines de pruebas.

---

# Completando **place_name** mediande **regex**, a partir de la colmna **place_with_parent_names**

Analizamos la cantidad de nulos de **place_name**

In [14]:
data2[["place_name", "place_with_parent_names"]].isnull().sum()

place_name                 23
place_with_parent_names     0
dtype: int64

Observamos una cantidad de nulos muy baja.
Podríamos optar por eliminar estos 23 registros nulos sin afectar considerablemente el dataset, pero si tenemos en cuenta que **place_with_parent_names** no posee nulos, podemos etraer los datos faltantes mediante la aplicación de finciones **lambda y regex**, allá vamos.

In [15]:
# Creamos los patrones a utilizar y los compilamos 
patern_barrio = '(\w+\W+)$'
regex_barrio = re.compile(patern_barrio)
regex_no_word = re.compile('\W+')

In [16]:
parent_names = data2.place_with_parent_names.astype(str)
barrios = parent_names.apply(lambda x: regex_barrio.findall(x))
barrios = barrios.apply(lambda x: regex_no_word.sub('', x[0]))
data2[["place_name"]] = barrios

In [17]:
data2[["place_name", "place_with_parent_names"]].isnull().sum()

place_name                 0
place_with_parent_names    0
dtype: int64

# Empleando distintas mask para filtrar nulos en las variables de precios, superficie y moneda

In [18]:
# Mascara de nulos para variables de interés
price_null                      = data2.price.isnull()
price_aprox_usd_null            = data2.price_aprox_usd.isnull()
price_aprox_local_currency_null = data2.price_aprox_local_currency.isnull()
price_usd_surf_total_null       = data2.price_usd_surf_total.isnull()
price_usd_surf_covered_null     = data2.price_usd_surf_covered.isnull()

surf_total_null                 = data2.surface_total_m2.isnull()
surf_covered_null               = data2.surface_covered_m2.isnull()

In [19]:
data2.currency.unique()

array(['USD', nan, 'ARS', 'PEN', 'UYU'], dtype=object)

Filtramos el dataset con una máscara donde **currency** posee valor **nan**, teniendo en cuenta los Features de interés

In [20]:
data2[['property_type','place_with_parent_names','currency','price','price_aprox_usd','price_aprox_local_currency','price_usd_surf_total','price_usd_surf_covered']][data2.currency.isna()].isnull().sum()

property_type                     0
place_with_parent_names           0
currency                      17354
price                         17353
price_aprox_usd               17353
price_aprox_local_currency    17353
price_usd_surf_total          17354
price_usd_surf_covered        17354
dtype: int64

In [21]:
data2[['property_type','place_with_parent_names','currency','price','price_aprox_usd','price_aprox_local_currency','price_usd_surf_total','price_usd_surf_covered']][data2.currency.isna()].isnull().sum()

property_type                     0
place_with_parent_names           0
currency                      17354
price                         17353
price_aprox_usd               17353
price_aprox_local_currency    17353
price_usd_surf_total          17354
price_usd_surf_covered        17354
dtype: int64

Podesmos observar que para los registros donde la variable "currency" en **nan**, las distintas variables de precios tampoco poseen un valor, por lo que procedemos a dropear los mismos.

In [22]:
print(f'Total registos antes de drop: {data2.shape[0]}')

data2.drop(data2[['property_type','place_with_parent_names','currency','price','price_aprox_usd','price_aprox_local_currency','price_usd_surf_total','price_usd_surf_covered']][data2.currency.isna()].index, inplace=True)

print(f'Total registos después de drop: {data2.shape[0]}\n')

print(f"Nulos sobre variales de interés posterior al drop:\n{data2[['property_type','place_with_parent_names','currency','price','price_aprox_usd','price_aprox_local_currency','price_usd_surf_total','price_usd_surf_covered']].isnull().sum()}")

Total registos antes de drop: 115678
Total registos después de drop: 98324

Nulos sobre variales de interés posterior al drop:
property_type                     0
place_with_parent_names           0
currency                          0
price                             0
price_aprox_usd                   0
price_aprox_local_currency        0
price_usd_surf_total          12866
price_usd_surf_covered        31716
dtype: int64


---
Observamos aquellos registros que están tazados en mondeas distintas al dolar 'USD', peso argentino 'ARS' o sin menoda de referencia 'nan'

In [23]:
data2[['property_type','place_with_parent_names','currency','price','price_aprox_usd','price_aprox_local_currency','surface_total_m2','price_usd_surf_total',
       'surface_covered_m2','price_usd_surf_covered']][(data2.currency == 'PEN')|(data2.currency == 'UYU')]

,property_type,place_with_parent_names,currency,price,price_aprox_usd,price_aprox_local_currency,surface_total_m2,price_usd_surf_total,surface_covered_m2,price_usd_surf_covered
50387,apartment,|Argentina|Capital Federal|,PEN,380000.0,117139.33,2.066865e+06,NaN,NaN,NaN,NaN
50388,apartment,|Argentina|Capital Federal|,PEN,950000.0,292848.33,5.167162e+06,NaN,NaN,NaN,NaN
107390,house,|Argentina|Mendoza|San Roque|,UYU,650000000.0,22980378.29,4.054773e+08,NaN,NaN,NaN,NaN


Al ser solo tres registros procindimos de los mismos mediante un drop

In [24]:
print(f'Total registos antes de drop: {data2.shape[0]}')
data2.drop(data2[(data2.currency == 'PEN')|(data2.currency == 'UYU')].index ,inplace=True)
print(f'Total registos después de drop: {data2.shape[0]}\n')

Total registos antes de drop: 98324
Total registos después de drop: 98321



---
# Ahora observemos las variables **surface_total_m2** y **surface_covered_m2**

In [25]:
data2[["surface_total_m2","surface_covered_m2"]].isnull().sum()

surface_total_m2      31464
surface_covered_m2    12861
dtype: int64

Aplicamos algunas máscaras

In [26]:
index_mask = data2[["surface_covered_m2"]][~surf_total_null & surf_covered_null].index

C:\Users\ciex\AppData\Local\Temp\ipykernel_53084\3300436376.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Completamos aquellos nulos de **surface_covered_m2** con los registros donde **surface_total_m2** posee un valor.

In [27]:
surf_covered_to_fill = data2.surface_total_m2.loc[index_mask]
surf_covered_to_fill

10        1514.0
15         164.0
66          38.0
142        380.0
168        195.0
           ...  
121199      52.0
121200      50.0
121202      52.0
121203      50.0
121205      50.0
Name: surface_total_m2, Length: 5383, dtype: float64

Verificamos si se aplicaron los cambios en cantidad de registros nulos

In [28]:
data2.surface_covered_m2.loc[index_mask] = surf_covered_to_fill

c:\Users\ciex\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
data2[["surface_total_m2","surface_covered_m2"]].isnull().sum()

surface_total_m2      31464
surface_covered_m2     7478
dtype: int64

---


In [30]:
data_nulls = pd.DataFrame(data.drop(['Unnamed: 0', 'operation', 'expenses', 'properati_url', 'image_thumbnail'], axis=1).isnull().sum().sort_values(ascending=False)[
                          data.drop(['Unnamed: 0', 'operation', 'expenses', 'properati_url', 'image_thumbnail'], axis=1).isnull().sum() > 0], columns=['Nulls'])
perc_nulls = pd.DataFrame(((data_nulls.values * 100)/data.shape[0]).round(3))
perc_nulls.columns = ['Percent']
perc_nulls = perc_nulls.applymap(lambda x: '{:.3f}%'.format(x))
data_nulls['Percent'] = perc_nulls.Percent.values


data2_nulls = pd.DataFrame(data2.isnull().sum().sort_values(ascending=False)[data2.isnull().sum() > 0], columns=['Nulls'])
perc_nulls = pd.DataFrame(((data2_nulls.values * 100)/data.shape[0]).round(3))
perc_nulls.columns = ['Percent']
perc_nulls = perc_nulls.applymap(lambda x: '{:.3f}%'.format(x))
data2_nulls['Percent'] = perc_nulls.Percent.values

px_bar = make_subplots(rows=1, cols=1)

px_bar.add_traces(go.Bar(name="Data Orignal", y=data_nulls.Nulls, x=data_nulls.index.values, text = data_nulls.Percent,
                         marker=dict(color = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], colorscale='Sunsetdark')))

px_bar.add_traces(go.Bar(name="Data2", y=data2_nulls.Nulls, x=data2_nulls.index.values, text = data2_nulls.Percent,
                         marker=dict(color = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], colorscale='Blugrn')))


px_bar.update_xaxes(tickangle=45, tickfont=dict(size=10))
px_bar.update_layout(title_text = 'Nulos Original / Nulos post Data Wrangling', font=dict(family="Consolas", size=17),
                     height=700, width=1300, template='plotly_dark')
px_bar.show()

# Finalmente exportamos data2 a un archivo csv

In [31]:
data2.to_csv('properatti2.csv', index=False)